In [8]:
import cv2
import os 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense , Dropout
import numpy as np
import pyttsx3
import matplotlib.pyplot as plt

In [ ]:
# root => where to pick the data/folder
root = 'MyData'

# dest => where to store the output data 
dest = 'D:\\Educational\Sign Language Recognition\MyData processed'

# looping over root directory's subdirectories
for enter in os.listdir(root):
    
    # specifying the final destination
    final = os.path.join(dest,enter)
    path = os.path.join(root,enter)
    
    # making the target/final directory
    os.makedirs(final)
    
    for item in os.listdir(path):
        
        # reading image from specified source
        image = cv2.imread(os.path.join(path,item),1)
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),2)
        th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
        ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        #Saving the image 
        name = os.path.join(final,item)
        cv2.imwrite(name ,res)

In [ ]:


sz = 128
train = r'C:\\Users\Dhruv\Downloads\train\train'
test = r'C:\\Users\Dhruv\Downloads\test\test'
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2, 2)))


# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2

# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


# Step 2 - Preparing the train/test data and training the model
classifier.summary()
# Code copied from - https://keras.io/preprocessing/image/
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train,
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(test,
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical')
checkpoint_cb = keras.callbacks.ModelCheckpoint("Best-Model-.h5",save_best_only =True)

In [ ]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=1284, # No of images in training set
        epochs=50,
        validation_data=test_set,
        validation_steps=426,
        callbacks = [checkpoint_cb])


In [13]:
SemodelImport = keras.models.load_model("C:\\Users\\Dhruv\\Downloads\\Best-Model-iit.h5")
label = [" ","A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
engine = pyttsx3.init()
vid = cv2.VideoCapture(0)
img1 = np.zeros((500,500,3),np.uint8)
out = ''
while(vid.isOpened()):
    ret,frame = vid.read()
#     cv2.rectangle(frame,(100,100),(400,400),(0,255,0),1)
    cv2.imshow("Image",frame)
    cv2.imshow("Text",img1)
    crop = frame[100:400, 100:400]
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),2)
    th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imshow("crop",res)
    k = cv2.waitKey(1)
    if k != -1:
        if k==27:
            break
        else:
            cv2.imwrite("test.jpg",res)
            image = tf.keras.preprocessing.image.load_img("test.jpg",grayscale=True ,target_size = (128,128))
            image_arr = tf.keras.preprocessing.image.img_to_array(image)
            image_arr = np.array([image_arr])
            pred = SemodelImport.predict_classes(image_arr)
            print(np.array(label)[pred])
            out = out + (str(np.array(label)[pred]))[2]
            cv2.putText(img1,out,(100,100),cv2.FONT_HERSHEY_SIMPLEX,0.6, (255,255,255), 1)
          
print(out)
vid.release()
cv2.destroyAllWindows() 
engine.setProperty('rate',100) #determine the rate of speech (optional)/ used for changing rate of speech
engine.say(out) #
engine.runAndWait() # the code will wait for the speech to complete

['W']
W


In [12]:
SemodelImport = keras.models.load_model("Best-Model-iit.h5")
out = ''
engine = pyttsx3.init()
label = [" ","A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
for i in range(1,6):
    image = tf.keras.preprocessing.image.load_img(str(i)+".jpg",grayscale=True ,target_size = (128,128))
    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    image_arr = np.array([image_arr])
    pred = SemodelImport.predict_classes(image_arr)
    print(np.array(label)[pred])
    out = out + (str(np.array(label)[pred]))[2]
engine.setProperty('rate',150) #determine the rate of speech (optional)/ used for changing rate of speech
engine.say(out) #
engine.runAndWait() 

['H']
['E']
['L']
['L']
['O']
